In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_value_counts = application_df['CLASSIFICATION'].value_counts()
class_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_type = class_value_counts.loc[class_value_counts > 1]
class_type

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_value_counts [class_value_counts <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = dummies_df['IS_SUCCESSFUL'].values
X = dummies_df.drop(columns='IS_SUCCESSFUL').values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y,test_size=.2)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 80
input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

/Users/karinagonzalez/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,081 (39.38 KB)

 Trainable params: 10,081 (39.38 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 314us/step - accuracy: 0.7083 - loss: 0.5886
Epoch 2/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 302us/step - accuracy: 0.7295 - loss: 0.5557
Epoch 3/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7342 - loss: 0.5523
Epoch 4/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7326 - loss: 0.5492
Epoch 5/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 306us/step - accuracy: 0.7332 - loss: 0.5443
Epoch 6/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 305us/step - accuracy: 0.7302 - loss: 0.5490
Epoch 7/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step - accuracy: 0.7246 - loss: 0.5554
Epoch 8/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 321us/step - accuracy: 0.7258 - loss: 0.5543
Epoch 9/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.7351 - loss: 0.5457
Epoch 10/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.7298 - loss: 0.5470
Epoch 11/200
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 313us/step - accuracy: 0.7352 - loss: 0.5427
Epoch 12/200
858/85

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 486us/step - accuracy: 0.7379 - loss: 0.5703
Loss: 0.570342481136322, Accuracy: 0.7379008531570435


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 30
units_3 = 20
input_features = len(X_train_scaled[0])

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=units_2, activation = "relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=units_3, activation = "relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,591 (25.75 KB)

 Trainable params: 6,591 (25.75 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model_2 = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 311us/step - accuracy: 0.6988 - loss: 0.5892
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 298us/step - accuracy: 0.7263 - loss: 0.5558
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 296us/step - accuracy: 0.7278 - loss: 0.5533
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step - accuracy: 0.7240 - loss: 0.5565
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - accuracy: 0.7336 - loss: 0.5474
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7270 - loss: 0.5505
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 297us/step - accuracy: 0.7298 - loss: 0.5487
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 294us/step - accuracy: 0.7300 - loss: 0.5503
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step - accuracy: 0.7343 - loss: 0.5452
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 295us/step - accuracy: 0.7326 - loss: 0.5481
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 292us/step - accuracy: 0.7365 - loss: 0.5396
Epoch 12/100
858/85

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 434us/step - accuracy: 0.7354 - loss: 0.5575
Loss: 0.5575084090232849, Accuracy: 0.7354227304458618


In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
units_1 = 80
units_2 = 30
input_features = len(X_train_scaled[0])

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=units_1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=units_2, activation = "tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn3.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model_3 = nn3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 1s 324us/step - accuracy: 0.7060 - loss: 0.5872
Epoch 2/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 0.7281 - loss: 0.5539
Epoch 3/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 281us/step - accuracy: 0.7284 - loss: 0.5518
Epoch 4/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7347 - loss: 0.5441
Epoch 5/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 301us/step - accuracy: 0.7302 - loss: 0.5484
Epoch 6/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step - accuracy: 0.7266 - loss: 0.5513
Epoch 7/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 315us/step - accuracy: 0.7337 - loss: 0.5458
Epoch 8/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.7335 - loss: 0.5484
Epoch 9/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.7268 - loss: 0.5527
Epoch 10/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.7269 - loss: 0.5509
Epoch 11/100
858/858 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step - accuracy: 0.7246 - loss: 0.5540
Epoch 12/100
858/85

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 427us/step - accuracy: 0.7369 - loss: 0.5486
Loss: 0.548560619354248, Accuracy: 0.7368804812431335


In [24]:
!pip install keras-tuner

In [25]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn4 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn4.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn4.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn4.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn4

In [26]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [27]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 08s]
val_accuracy: 0.7361515760421753

Best val_accuracy So Far: 0.7380466461181641
Total elapsed time: 00h 03m 21s


In [28]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 615us/step - accuracy: 0.7380 - loss: 0.5486
Loss: 0.5485810041427612, Accuracy: 0.7380466461181641
215/215 - 0s - 657us/step - accuracy: 0.7380 - loss: 0.5721
Loss: 0.5720921754837036, Accuracy: 0.7380466461181641
215/215 - 0s - 628us/step - accuracy: 0.7378 - loss: 0.5502
Loss: 0.5501610040664673, Accuracy: 0.7377551198005676


In [29]:
nn3.save('AlphabetSoupCharity_Optimization.h5')